# **IMPLEMENTATION OF THE DIMENSIONALITY REDUCTION**

In [35]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [36]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.644292,0.489853,0.433547,0.002361,0.008748,0.043237,0.002444,0.008546,0.038236,0.032638,...,0.475511,0.350991,0.147940,0.437030,0.515200,0.267793,0.491741,0.079377,0.798251,0.470687
1,0.639209,0.491795,0.438240,0.000877,0.012824,0.019839,0.000596,0.012749,0.021157,0.028466,...,0.594891,0.201676,0.070666,0.521131,0.496283,0.133774,0.853429,0.077606,0.798487,0.472842
2,0.639827,0.490266,0.443269,0.002310,0.017058,0.010528,0.001740,0.018464,0.011266,0.030654,...,0.726616,0.304605,0.122488,0.434094,0.588950,0.550707,0.906041,0.075533,0.798722,0.475441
3,0.639587,0.486900,0.438359,0.001954,0.008596,0.004662,0.001450,0.008767,0.005349,0.030654,...,0.721515,0.442374,0.264192,0.475464,0.493554,0.820538,0.257822,0.075675,0.799600,0.476168
4,0.638314,0.491715,0.442319,0.000931,0.009945,0.004759,0.000839,0.010331,0.004779,0.028765,...,0.558768,0.324393,0.153612,0.556466,0.561271,0.847339,0.192392,0.076067,0.801776,0.478054


In [37]:
y.head()

,0
0,4
1,4
2,4
3,4
4,4


In [38]:
y.nunique()

6

In [39]:
#ADD EDA steps
# Display first few rows of the dataset
print(df.head())

# Summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Data type information
print(df.info())



        0         1         2         3         4         5         6    \
0  0.644292  0.489853  0.433547  0.002361  0.008748  0.043237  0.002444   
1  0.639209  0.491795  0.438240  0.000877  0.012824  0.019839  0.000596   
2  0.639827  0.490266  0.443269  0.002310  0.017058  0.010528  0.001740   
3  0.639587  0.486900  0.438359  0.001954  0.008596  0.004662  0.001450   
4  0.638314  0.491715  0.442319  0.000931  0.009945  0.004759  0.000839   

        7         8         9    ...       551       552       553       554  \
0  0.008546  0.038236  0.032638  ...  0.475511  0.350991  0.147940  0.437030   
1  0.012749  0.021157  0.028466  ...  0.594891  0.201676  0.070666  0.521131   
2  0.018464  0.011266  0.030654  ...  0.726616  0.304605  0.122488  0.434094   
3  0.008767  0.005349  0.030654  ...  0.721515  0.442374  0.264192  0.475464   
4  0.010331  0.004779  0.028765  ...  0.558768  0.324393  0.153612  0.556466   

        555       556       557       558       559       560  
0  0

In [40]:
from sklearn.preprocessing import LabelEncoder

y_list = y.tolist()

le = LabelEncoder()

y_encoded = le.fit_transform(y_list)

y = pd.Series(y_encoded)

In [41]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

In [42]:
#Normalization
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
scaler = MinMaxScaler()

# Fit the scaler to your data and transform it
df[numeric_features] = scaler.fit_transform(df[numeric_features])


In [43]:
#Standardization numeric values
# numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# scaler = StandardScaler()
# df[numeric_features] = scaler.fit_transform(df[numeric_features])


In [44]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [45]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Model training using Random Forest
model = GaussianNB()
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

from sklearn.metrics import confusion_matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7307953772943576
Confusion Matrix:
 [[183  46  18   0   0   0]
 [  5 182  13   0   0   0]
 [  9  41 156   0   0   0]
 [  0   4   0 192  63   3]
 [  0   4   0  24 246   2]
 [  0   3   0 161   0 116]]


In [46]:
# Define a Pipeline with Gaussian Naive Bayes and train it on the full dataset
start_time = time.time()
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(X_train, y_train)
y_pred_full = classifier_pipeline_full.predict(X_test)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)
print("Time: ",full_features_time)
print("Accuracy:", accuracy_full)

Time:  0.2790508270263672
Accuracy: 0.7307953772943576


In [55]:
# SOLUTION -  K-Means for dimensionality reduction
n_clusters = 50


kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df.iloc[:, selected_features_indices]

In [57]:
#After reducing the dimensions

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42)



start_time = time.time()1
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(X_train, y_train)
y_pred_full = classifier_pipeline_full.predict(X_test)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(y_test, y_pred_full)
print("Time: ",full_features_time)
print("Accuracy:", accuracy_full)

Time:  0.015409231185913086
Accuracy: 0.8783140720598233
